In [1]:
import pandas as pd
import sys
import pickle

sys.path.insert(0, "./../../src/utils/")

from utils import make_tree, find_most_similar_artwork

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from tqdm import tqdm

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec

from utils import get_scores

In [2]:
# https://stackoverflow.com/questions/45441909/how-to-add-a-fixed-width-border-to-subplot


def get_subplot_border(ax, width=7, color="blue"):

    # Convert bottom-left and top-right to display coordinates
    x0, y0 = ax.transAxes.transform((0, 0))
    x1, y1 = ax.transAxes.transform((1, 1))

    # Adjust margins
    x0 -= width
    x1 += width
    y0 -= width
    y1 += width

    # Convert back to Axes coordinates
    x0, y0 = ax.transAxes.inverted().transform((x0, y0))
    x1, y1 = ax.transAxes.inverted().transform((x1, y1))

    rect = plt.Rectangle((x0, y0), x1-x0, y1-y0,
                         color=color,
                         transform=ax.transAxes,
                         zorder=-1)

    return rect


def plot_before_after(drawing_name, similar_artworks_before, similar_artworks_after, used_in, uid2path_df, actual_close_artwork):
    
    drpath = uid2path_df.loc[uid2path_df['uid'] == drawing_name, 'path'].iloc[0]

    # fig = plt.figure(1, figsize=(20,11.69))
    fig = plt.figure(1, figsize=(11.69,8.27), dpi=300)
    # fig = plt.figure(1, figsize=(16.53,11.69))

    outer = gridspec.GridSpec(1, 3, wspace=0.2, hspace=0.2)
    drimage = img.imread(data_folder + drpath)

    inner_center = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer[1], wspace=0.1, hspace=0.1)

    ax = plt.Subplot(fig, inner_center[0])
    ax.imshow(drimage)
    ax.set_title(drawing_name)
    ax.axis(False)
    fig.add_subplot(ax)


    inner_left = gridspec.GridSpecFromSubplotSpec(5, 3, subplot_spec=outer[0], wspace=0.1, hspace=0.1)
    inner_right = gridspec.GridSpecFromSubplotSpec(5, 3, subplot_spec=outer[2], wspace=0.1, hspace=0.1)


    for ind, sim_art_before in enumerate(similar_artworks_before):
        artname = sim_art_before[0]
        artpath = uid2path_df.loc[uid2path_df['uid'] == artname, 'path'].iloc[0]
        artimage = img.imread(data_folder + artpath)
        ax = plt.Subplot(fig, inner_left[ind])
        if artname == actual_close_artwork:
            border = get_subplot_border(ax)
            fig.patches.append(border)
        ax.imshow(artimage)
        ax.set_title(artname, fontsize=5)
        ax.axis(False)
        fig.add_subplot(ax)

    for ind, sim_art_after in enumerate(similar_artworks_after):
        artname = sim_art_after[0]
        artpath = uid2path_df.loc[uid2path_df['uid'] == artname, 'path'].iloc[0]
        artimage = img.imread(data_folder + artpath)
        ax = plt.Subplot(fig, inner_right[ind])
        if artname == actual_close_artwork:
            border = get_subplot_border(ax)
            fig.patches.append(border)
        ax.imshow(artimage)
        ax.set_title(artname, fontsize=5)
        ax.axis(False)
        fig.add_subplot(ax)


    subgrid_titiles = ["Before fine-tuning", "Drawing used in {}".format(used_in), "After fine-tuning"]

    for ind, title in enumerate(subgrid_titiles):
        ax_ = plt.Subplot(fig, outer[ind])
        ax_.set_title(title)
        ax_.axis('off')
        fig.add_subplot(ax_)    
        
        
def save_before_after_finetuning_images(drawings_artwork_pairs: pd.DataFrame, uid2path_df: pd.DataFrame, drawings_embeddings_before: np.ndarray, drawings_embeddings_after: np.ndarray, artworks_tree_before, artworks_tree_after, artworks_names_before: list[str], artworks_names_after: list[str], full_path: str):
    with PdfPages(full_path) as pdf:
        for pair_set in tqdm(["train", "val", "test"]):
            subset_df = drawings_artwork_pairs[drawings_artwork_pairs["set"] == pair_set].sort_values(by=['artwork_id'])

            plt.figure() 
            plt.axis('off')
            plt.text(0.5,0.5,pair_set,ha='center',va='center')
            pdf.savefig()
            plt.close()

            for rid, row in subset_df.iterrows():
                drawing_name = row["drawing_id"]
                actual_close_artwork = row["artwork_id"]
                used_in = row["set"]
                similar_artworks_before = find_most_similar_artwork(drawing_name, artworks_tree_before, drawings_embeddings_before, artworks_names_before, n=15)
                similar_artworks_after = find_most_similar_artwork(drawing_name, artworks_tree_after, drawings_embeddings_after, artworks_names_after, n=15)

                plot_before_after(drawing_name, similar_artworks_before, similar_artworks_after, pair_set, uid2path_df, actual_close_artwork)

                pdf.savefig(bbox_inches="tight")
                plt.close()

In [3]:
data_folder = "./../../data/"
uid2path = "./../../data/text_data/uid2path.pkl"

drawings_embeddings_path_before = "./../../data/embeddings/drawings_learning_resnext-101_avg_280_epoch_0_initial.npy"
artworks_embeddings_path_before = "./../../data/embeddings/famous_artworks_resnext-101_avg_280_epoch_0_initial.npy"

drawings_embeddings_path_after = "./../../data/embeddings/drawings_learning_resnext-101_avg_280_epoch_42_29-05-2022_00-20-55.npy"
artworks_embeddings_path_after = "./../../data/embeddings/famous_artworks_resnext-101_avg_280_epoch_42_29-05-2022_00-20-55.npy"

In [4]:
draw_embeds_bfr = np.load(drawings_embeddings_path_before, allow_pickle=True)
arts_embed_bfr = np.load(artworks_embeddings_path_before, allow_pickle=True)

draw_embeds_afr = np.load(drawings_embeddings_path_after, allow_pickle=True)
arts_embed_afr = np.load(artworks_embeddings_path_after, allow_pickle=True)

with open(uid2path, "rb") as outfile:
    uid2path_mapping = pickle.load(outfile)
    
draw_art_pairs_df = pd.read_csv("./../../data/model_learning/drawing_artwork_pair_train_test_split.csv")

In [5]:
arts_names_bfr = arts_embed_bfr[:, 0].tolist()
arts_names_afr = arts_embed_afr[:, 0].tolist()

arts_tree_bfr = make_tree(arts_embed_bfr)
arts_tree_afr = make_tree(arts_embed_afr)

In [6]:
save_before_after_finetuning_images(draw_art_pairs_df, uid2path_mapping,
                                    draw_embeds_bfr, draw_embeds_afr,
                                    arts_tree_bfr, arts_tree_afr,
                                    arts_names_bfr, arts_names_afr,
                                    full_path='./../../data/intermediate_storage/before_after_training_comparision_first_training.pdf')

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [07:11<00:00, 143.69s/it]
